In [1]:
import traceback
import opensim as osim
from connections import AWS
import xml.etree.ElementTree as ET
from biomech.processing.trc import *

In [2]:
""" INITIALIZE AWS CONNECTION """
aws_connection = AWS()
aws_connection.connect()

[AWS]: Port 5433 is free.
[AWS]: Connected to RDS endpoint.


$\textbf{Inverse Kinematics: Ball Release Times}$

- Conservative window: BR - 0.217 seconds
- When IK is run, add 0.25 seconds on either side for later trimming

In [3]:
from biomech.algorithms import diff_three_point

In [4]:
# load a .mot file (e.g., results from an IK run)
def load_mot_file(path: str) -> pd.DataFrame:
    return pd.read_csv(path, delim_whitespace=True, skiprows=10)

In [ ]:
# load all subject info
subject_info = aws_connection.load_subject_info()

# load all (filtered) trc files
s3_objects = aws_connection.list_s3_objects(prefix='biomechanics/subjects/')
trc_files = [obj for obj in s3_objects if obj.endswith('.trc') and '_static' not in obj and '_filtered' in obj]

In [ ]:
# initialize error log, ball release times
ball_release_times = []
error_log = []

# iterate through subjects
for subject in subject_info['subject_id'].unique():
    
    try:
        # get subject info (ID -- string; throwing hand)
        subject_id = str(subject)
        throwing_hand = subject_info[subject_info['subject_id'] == subject]['throws'].values[0]

        # list all subject trials (in trc_processed) & iterate through
        subject_trials = [t for t in trc_files if f'{subject_id}' in t]
        
        if len(subject_trials) > 0:
            print(f'Processing subject {subject_id} with {len(subject_trials)} trials...')

            # iterate through trials
            for trial in subject_trials:
                # load trial TRC file
                trial_trc_bytes = aws_connection.load_s3_object(trial, return_info=False)
                trial_trc_body = parse_trc_body(trial_trc_bytes, filter_markers=False, adjust_time=False)

                # save TRC file to local dir
                write_to_trc('trial.trc', trial_trc_body, throwing_hand, filter_markers=False)

                """ BALL RELEASE IDENTIFICATION """
                # identify ball release (peak hand speed in x-direction + 5 frames)
                hand_markers = ['X7', 'Y7', 'Z7']
                trial_hand_speed = diff_three_point(trial_trc_body[hand_markers].values)
                ball_release_index = trial_hand_speed[:, 0].argmax() + 5                            # peak speed (x-direction) + 5 frames
                ball_release_time = trial_trc_body['Time'].iloc[ball_release_index]                  # time of ball release

                # save ball release time
                trial_id = trial.split('/')[-1].split('_filtered')[0]
                ball_release_times.append({
                    'trial_id': trial_id,
                    'ball_release_time': ball_release_time
                })

            print(f'Completed processing for subject {subject_id}.')

    except Exception as e:
        print(f"Error processing subject {subject_id}: {e}")

        traceback.print_exc()

        # log error
        error_log.append({
            'subject_id': subject_id,
            'error': str(e)
        })

        continue

# save ball release times to S3
ball_release_df = pd.DataFrame(ball_release_times)
aws_connection.upload_to_s3(
    ball_release_df,
    s3_key='biomechanics/subjects/summary/ball_release_times.csv',
)

$\textbf{Close AWS Connection}$

In [12]:
aws_connection.close()

[AWS]: Database connection closed.
[AWS]: SSH tunnel stopped.
